# Max bond dimension

In [ ]:
from __future__ import annotations

import random
from typing import Sequence
import numpy as np
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit.quantum_info import Statevector
from qiskit.circuit.random import random_circuit
from qiskit_aer import AerSimulator
from ttz.mps import MPS

## Qiskit

In [ ]:
def walk_tensors(tensors: Sequence, bond_dims: list[int]):
    for t in tensors:
        if isinstance(t, np.ndarray):
            bond_dims.append(max(t.shape))
        else:
            walk_tensors(t, bond_dims)


def generate_custom_random_circuit(num_qubits: int, depth: int, allowed_gates: list[str]):
    qc = QuantumCircuit(num_qubits)

    for _ in range(depth):
        gate_type = random.choice(allowed_gates)

        if gate_type in ['x', 'y', 'z', 'h', 't', 's']:
            q = random.randint(0, num_qubits - 1)
            eval(f'qc.{gate_type}(q)')

        elif gate_type in ['rx', 'ry', 'rz']:
            q = random.randint(0, num_qubits - 1)
            theta = random.uniform(0, 2 * np.pi)
            eval(f'qc.{gate_type}(theta, q)')

        elif gate_type in ['cx', 'cy', 'cz']:
            if num_qubits < 2:
                continue
            control, target = random.sample(range(num_qubits), 2)
            eval(f'qc.{gate_type}(control, target)')

    return qc


allowed = ['h', 'rx', 'cx']
random.seed(1234)
rand_circ = generate_custom_random_circuit(num_qubits=10, depth=30, allowed_gates=allowed)
rand_circ2 = rand_circ.copy()
rand_circ2.save_matrix_product_state()

sim = AerSimulator(method='matrix_product_state')
#tcirc = transpile(rand_circ2, sim)
result = sim.run(rand_circ2).result()
mps = result.data(0)['matrix_product_state']

bond_dims = []
walk_tensors(mps, bond_dims)
print(max(bond_dims), bond_dims)

## TTZ

In [ ]:
def qc2mps(qc: QuantumCircuit) -> MPS:
    mc = MPS(qc.num_qubits)
    for inst in qc.data:
        if inst.operation.num_qubits == 1:
            idx0 = inst.qubits[0]._index
            if len(inst.params) > 0:
                eval(f'mc.{inst.name}({inst.params[0]}, {idx0})')
            else:
                eval(f'mc.{inst.name}({idx0})')
        elif inst.operation.num_qubits == 2:
            idx0 = inst.qubits[0]._index
            idx1 = inst.qubits[1]._index
            eval(f'mc.{inst.name}({idx0}, {idx1})')
    return mc


mps2 = qc2mps(rand_circ)
print(np.allclose(Statevector(rand_circ), mps2.state_vector()))

print('max bond dimension =', max([lam.shape[0] for lam in mps2._lambdas]))